In [ ]:
# Setup
import pandas as pd
import os
import pymysql
pymysql.install_as_MySQLdb()
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
from sqlalchemy import create_engine, inspect, Column, Integer, String
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy.sql import func
from flask import Flask, jsonify

In [ ]:
# Step 1 - Climate Analysis and Exploration

In [ ]:
# To begin, use Python and SQLAlchemy to do basic climate analysis and data exploration of your climate database.
# All of the following analysis should be completed using SQLAlchemy ORM queries, Pandas, and Matplotlib.
# Use the provided [starter notebook](climate_starter.ipynb) and [hawaii.sqlite](Resources/hawaii.sqlite)
# files to complete your climate analysis and data exploration.

In [ ]:
# Choose a start date and end date for your trip.
# Make sure that your vacation range is approximately 3-15 days total.

In [ ]:
# Use SQLAlchemy `create_engine` to connect to your sqlite database.

# Connect to database
engine = create_engine('sqlite:///hawaii.sqlite')

# Create tables in database
Base.metadata.create_all(engine)

# Start session
session = Session(bind=engine)

In [ ]:
# Use SQLAlchemy `automap_base()` to reflect your tables into classes and
# save a reference to those classes called `Station` and `Measurement`.

# Declare Base using 'automap_base()'
Base = automap_base()

# Use Base class to reflect database tables
Base.prepare(engine, reflect=True)

# Use Base class to reflect database tables
Base.prepare(engine, reflect=True)

In [ ]:
### Precipitation Analysis

# Design a query to retrieve the last 12 months of precipitation data.
# Select only the `date` and `prcp` values.

latestDate = (session.query(Measurement.date)
                     .order_by(Measurement.date.desc())
                     .first())
latestDate = list(np.ravel(latestDate))[0]
latestDate = dt.datetime.strptime(latestDate, '%Y-%m-%d')

latestYear = int(dt.datetime.strftime(latestDate, '%Y'))
latestMonth = int(dt.datetime.strftime(latestDate, '%m'))
latestDay = int(dt.datetime.strftime(latestDate, '%d'))

yearBefore = dt.date(latestYear, latestMonth, latestDay) - dt.timedelta(days=365)

rainData = (session.query(Measurement.date, Measurement.prcp)
                  .filter(Measurement.date > yearBefore)
                  .order_by(Measurement.date)
                  .all())

rainTable = pd.DataFrame(rainData)
rainTable = rainTable.set_index('date')rainTable = rainTable.sort_index(ascending=True)

In [ ]:
# Load the query results into a Pandas DataFrame and set the index to the date column.
# Sort the DataFrame values by `date`.

tobs_df = pd.DataFrame(tobs_query, columns=['date','station','tobs'])

#tobs_df.set_index('date', inplace=True)
#tobs_df.head()

tobs_only = tobs_df['tobs']

In [ ]:
# Plot the results using the DataFrame `plot` method.

  ![precipitation](Images/precipitation.png)

In [ ]:
### Station Analysis

HI_stations = Base.classes.stations

list(HI_stations.__table__.columns)

In [ ]:
# Design a query to calculate the total number of stations.

stations_count = (session
                  .query(HI_stations.station)
                  .count())
stations_count

In [ ]:
# Design a query to find the most active stations.
# List the stations and observation counts in descending order.
# Which station has the highest number of observations?
active_stations = (session
                   .query(HI_measurements.station, func.count(HI_measurements.date))
                   .order_by(func.count(HI_measurements.date).desc())
                   .group_by(HI_measurements.station)
                   .all())
active_stations

In [ ]:
# Design a query to retrieve the last 12 months of temperature observation data (tobs).
# Filter by the station with the highest number of observations.
tobs_query = (session
              .query(HI_measurements.date, 
                     HI_measurements.station,
                     HI_measurements.tobs)
              .filter(HI_measurements.date <= '2017-08-23')
              .filter(HI_measurements.date >= '2016-08-23')
              .order_by(HI_measurements.date)
              .all())

for row in tobs_query:
    print(row)

In [ ]:
# Plot the results as a histogram with `bins=12`.
plt.hist(tobs_only, bins=12)
plt.xlabel('tobs')
plt.ylabel('Frequency')
plt.title('tobs frequency of most active train USC00519281 from 8/23/2016 - 8/23/2017')
plt.grid()
plt.show()

![station-histogram](Images/station-histogram.png)

In [ ]:
# Step 2 - Climate App

In [ ]:
# Now that you have completed your initial analysis,
# design a Flask API based on the queries that you have just developed.

### Routes
    # `/`
    # Home page.
    # List all routes that are available.

In [ ]:
# `/api/v1.0/precipitation`
# Convert the query results to a Dictionary using `date` as the key and `prcp` as the value.
# Return the JSON representation of your dictionary.

app = Flask(__name__)

@app.route('/api/v1.0/precipitation') 
def precipitation():
    print("Server received request for 'Precipitation' page...")
    return "Here are the dates and temperature observations from last year!"

    last_year_temp = (session
                      .query(HI_measurements.tobs)
                      .filter(HI_measurements.date <= '2017-08-23')
                      .filter(HI_measurements.date >= '2016-08-23')
                      .order_by(HI_measurements.date)
                      .all())

    # Convert the query results to a Dictionary using date as the key and tobs as the value
    dates_and_temp = []
    for date in last_year_temp:
        date_dict ={}
        date_dict['date'] = HI_measurements.date
        date_dict['temperature'] = HI_measurements.tobs
        dates_and_temp.append(date_dict)
    
# Return the json representation of your dictionary
    return jsonify(dates_and_temp)

In [ ]:
# `/api/v1.0/stations`
# Return a JSON list of stations from the dataset.

@app.route('/api/v1.0/stations') 
def stations():
    print("Server received request for 'Stations' page...")
    return "Here is the list of stations from the dataset!"

    stations_query = (session
                      .query(HI_stations.stations)
                      .all())
    
    return jsonify(stations_query)

In [ ]:
# `/api/v1.0/tobs`
# query for the dates and temperature observations from a year from the last data point.
# Return a JSON list of Temperature Observations (tobs) for the previous year.

@app.route('/api/v1.0/tobs') 
def tobs():
    print("Server received request for 'Temperature Observations' page...")
    return "Here is the list of Temperature Observations (tobs) for the previous year!"
    
    tobs_last_year = (session
                      .query(HI_measurements.tobs)
                      .filter(HI_measurements.date <= '2017-08-23')
                      .filter(HI_measurements.date >= '2016-08-23')
                      .order_by(HI_measurements.tobs)
                      .all())
    
    return jsonify(tobs_last_year)

In [ ]:
# `/api/v1.0/<start>` and `/api/v1.0/<start>/<end>`
# Return a JSON list of the minimum temperature, the average temperature, and the max temperature for 
# a given start or start-end range.
# When given the start only, calculate `TMIN`, `TAVG`, and `TMAX` for all dates greater than and equal
# to the start date.
# When given the start and the end date, calculate the `TMIN`, `TAVG`, and `TMAX` for dates between
# the start and end date inclusive.

@app.route('/api/v1.0/<start>)') 
def tobs_start():
    print("Server received request for 'Min Temp, Avg Temp, Max Temp of a given start' page...")
    return "Here is the list of Temperature information for the a given date!"
    
    start = HI_measurements.date <= '2010-01-01'
    end = HI_measurements.date >= '2017-08-23'

    # When given the start only, calculate TMIN, TAVG, and TMAX for all dates greater 
    # than and equal to the start date
    tobs_start_only = (session
                       .query(HI_measurements.tobs)
                       .filter(HI_measurements.date.between(start, '2017-08-23'))
                       .all())
    
    tso_df = pd.DataFrame(tobs_start_only)
    
    tmin = tso_df.min()
    tmax = tso_df.max()
    tavg = tso_df.avg()
    
    return jsonify(tmin, tmax, tavg)

@app.route('/api/v1.0/<start>/<end>') 
def tobs_given_range():
    
    print("Server received request for 'Min Temp, Avg Temp, Max Temp of a given start or date range' page...")
    return "Here is the list of Temperature information for the a given date range!"
    
    # When given the start and the end date, calculate the TMIN, TAVG, and TMAX for dates between the start 
    # and end date inclusive.
    tobs_given_range = (session
                        .query(HI_measurements.tobs)
                        .filter(HI_measurements.date.between(start, end))
                        .all())
    
    # Convert to Dataframe to calculate TMIN, TAVG, TMAX
    tgr_df = pd.DataFrame(tobs_given_range)
    
    t2min = tgr_df.min()
    t2max = tgr_df.max()
    t2avg = tgr_df.avg()
    
    return jsonify(t2min, t2max, t2avg)